# __Group Project 3 - Team Terence Gorman__
<br>


### <span style = "color:blue">__Team Members: Marc Bonnot (*Team Manager*), Ce (Faye) Fang, Dojun Kim, Kimberley Fan, Jane Wang, Qianhao (Harry) Zhong__
<br>


## **1. Project Topic:**

> 
<br>


## **2. Project Goal:**

>
<br>


## **3. [A] Choice of a population, with supporting expenditure data:**

>
<br>


## **4. Data Description:**

>    
<br>


## **5. Project Deliverables:**

### *Packages Import*:

In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 321.7 kB/s eta 0:00:00a 0:00:01
  Using cached ConsumerDemands-0.4.2.dev0-py2.py3-none-any.whl (12 kB)
  Using cached python_gnupg-0.5.2-py2.py3-none-any.whl (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 15.6 MB/s eta 0:00:0000:0100:01
  Using cached xarray-2024.2.0-py3-none-any.whl (1.1 MB)
  Using cached dvc-3.48.4-py3-none-any.whl (450 kB)
  Using cached iterative_telemetry-0.0.8-py3-none-any.whl (10 kB)
  Using cached rich-13.7.1-py3-none-any.whl (240 kB)
  Using cached voluptuous-0.14.2-py3-none-any.whl (31 kB)
  Using cached configobj-5.0.8-py2.py3-none-any.whl (36 kB)
  Using cached dvc_studio_client-0.20.0-py3-none-any.whl (16 kB)
  Using cached dpath-2.1.6-py3-none-any.whl (17 kB)
  Using cached tomlkit-0.12.4-py3-none-any.whl (37 kB)
  Using cached platformdirs-3.11.0-py3-none-any.whl (17 kB)
  Using cached dvc_render-1.0.1-py3-none-any.whl (22 kB)
  Using cached grandalf-0.8-py3-none-any.whl (41 k

In [2]:
from eep153_tools.sheets import read_sheets
import numpy as np
import pandas as pd

### *DATA Import, DATA CLEAN*:

In [3]:
InputFiles = {"Expenditures": ("1P7OQG9cmBSS4dY6ZPI1HOPHW9shCTSkguDVnUCuh3Is", "Food Expenditures Translated"),
              'Prices':('1P7OQG9cmBSS4dY6ZPI1HOPHW9shCTSkguDVnUCuh3Is','Food Prices Translated'),
              'HH Characteristics':('1P7OQG9cmBSS4dY6ZPI1HOPHW9shCTSkguDVnUCuh3Is','Household Characteristics'),
              'FCT':('1P7OQG9cmBSS4dY6ZPI1HOPHW9shCTSkguDVnUCuh3Is','FCT Translated'),
              'RDI':('1yVLriVpo7KGUXvR3hq_n53XpXlD5NmLaH1oOMZyV0gQ','RDI')}

In [4]:
def get_clean_sheet(key,sheet=None):
    df = read_sheets(key,sheet=sheet)
        
    df.columns = [c.strip() for c in df.columns.get_level_values(0).tolist()]

    df = df.loc[:,~df.columns.duplicated(keep='first')]   

    df = df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)

    df = df.loc[~df.index.duplicated(), :]

    return df

In [5]:
# Get expenditures...
Expenditures = get_clean_sheet(InputFiles['Expenditures'][0],
                    sheet=InputFiles['Expenditures'][1])

if 'm' not in Expenditures.columns:
    Expenditures['m'] = 1

Expenditures = Expenditures.set_index(['i','t','m'])
Expenditures.columns.name = 'j'

Expenditures = Expenditures.apply(lambda x: pd.to_numeric(x,errors='coerce'))
Expenditures =Expenditures.replace(0,np.nan)

# Get HH characteristics...
HH_characteristics = get_clean_sheet(InputFiles['HH Characteristics'][0],
                    sheet=InputFiles['HH Characteristics'][1])

if 'm' not in HH_characteristics.columns:
    HH_characteristics['m'] = 1

HH_characteristics = HH_characteristics.set_index(['i','t','m'])
HH_characteristics.columns.name = 'k'

HH_characteristics = HH_characteristics.apply(lambda x: pd.to_numeric(x,errors='coerce'))

# Get prices
price_url = 'https://docs.google.com/spreadsheets/d/1P7OQG9cmBSS4dY6ZPI1HOPHW9shCTSkguDVnUCuh3Is/edit#gid=1352592273'

prices = read_sheets(price_url,sheet='Food Prices Translated',nheaders=2)
prices.columns.names=['t', 'm']
##since there are multiple prices for food in various sizes, we choose to use average price 
prices.groupby(level='j').mean()
prices=prices.transpose()
if 'm' not in prices.columns:  # Supply "market" indicator if missing
    prices['m'] = 1
prices = prices.apply(lambda x: pd.to_numeric(x,errors='coerce'))
prices = prices.replace(0,np.nan)


###fct

fct = get_clean_sheet(InputFiles['FCT'][0],
                    sheet=InputFiles['FCT'][1])

########################################
fct = fct.set_index('j')
fct.columns.name = 'n'

fct = fct.apply(lambda x: pd.to_numeric(x,errors='coerce'))

################## RDI, if available (consider using US) #####################
rdi = get_clean_sheet(InputFiles['RDI'][0],
                    sheet=InputFiles['RDI'][1])
rdi = rdi.set_index('n')
rdi.columns.name = 'k'

#### To analyze the province/region effect on food expenditure, we choose to add indicator variables (dummy variables) of locations in Household characteristics 

In [27]:
###add region indicator variables into HH_characteristics
dummies=pd.get_dummies(HH_characteristics.index.get_level_values(2), dtype=float)

####Drop the NA column
####To aviod multicollinearity when we do regression, we need to drop one dummy variable--"Bocas Del Toro"
dummies=dummies.drop(columns=["#N/A", "Bocas Del Toro"])

##add dummies variables to HH_chracteristics dataset
HH_characteristics=HH_characteristics.reset_index().join(dummies).set_index(["i", "t", "m"])

#### To analyze the event effect on food expenditure, we will analyze the food expenditure/demand before the event and after event. The following function is to split the dataset based on a time (event) 

In [ ]:
###function to split the dataset based on a time (before time t, after time t)
def split_data_t(time, data):
    before_t=data[data.index.get_level_values('t') <= time]
    after_t=data[data.index.get_level_values('t') >time]
    return before_t, after_t

## [B] Nutritional Content of Different Foods

## [B] Nutritional Adequacy of Diet

## [A] Estimate Demand System

## [C] Counterfactual Experiments

## Visualization